In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from tokenizer import BertTokenizerWithMapping
from models.mlp import BertMTL, BertClassifier
from models.params import MTLParams
from copy import deepcopy
import os 
from flask import Flask, request, redirect, url_for, render_template
from azure_search.bing_utils import bing_wiki_search, get_wiki_docs
from itertools import chain
import numpy as np 


bert_dir = 'bert-base-uncased'
evi_finder_loc = './trained_models/fever/evidence_token_identifier.pt'
cls_loc = 'trained_models/fever/evidence_classifier.pt'
classes = ["SUPPORTS", "REFUTES"]
device = torch.device('cpu')
top = 4

/home/assassin/anaconda3/envs/pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/assassin/anaconda3/envs/pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/assassin/anaconda3/envs/pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/assassin/anaconda3/envs/pytorch

In [3]:
print("Loading model")
tokenizer = BertTokenizerWithMapping.from_pretrained(bert_dir)
max_length = 512
use_half_precision = False

mtl_params = MTLParams(dim_cls_linear=256, num_labels=2, dim_exp_gru=128)

evi_finder = BertMTL(bert_dir=bert_dir,
                     tokenizer=tokenizer,
                     mtl_params=mtl_params,
                     use_half_precision=False)
evi_finder.load_state_dict(torch.load(evi_finder_loc, map_location=device))

cls = BertClassifier(bert_dir=bert_dir,
                     pad_token_id=tokenizer.pad_token_id,
                     cls_token_id=tokenizer.cls_token_id,
                     sep_token_id=tokenizer.sep_token_id,
                     num_labels=mtl_params.num_labels,
                     max_length=max_length,
                     mtl_params=mtl_params,
                     use_half_precision=False)
cls.load_state_dict(torch.load(cls_loc, map_location=device))

Loading model


<All keys matched successfully>

In [4]:
# query = 'microsoft is a chinese company'
app = Flask(__name__)
# import jinja2
# env = jinja2.Environment()
# env.globals.update(zip=zip)
# app.jinja_env.filters['zip'] = zip

In [5]:
def clean(query):
    query = query.strip().lower()
    return query


@app.route('/', methods=['GET', 'POST']) 
def main_page():
    if request.method == 'POST':
        query = request.form['query']
        query = clean(query)
        return redirect(url_for('prediction', query=query))
    return render_template('index.html')


def preprocess(query, docs):
    query = query.split()
    tokenized_q, tokenized_q_token_slides = tokenizer.encode_docs([[query]])
    tokenized_q = tokenized_q[0]
    tokenized_q_token_slide = tokenized_q_token_slides[0]
    docs_clean = [[list(chain.from_iterable(s.split() + ['.'] for s in d.lower().split('.')))] 
                  for d in docs]
    docs = deepcopy(docs_clean)
    tokenized_docs, tokenized_doc_token_slides = tokenizer.encode_docs(docs)
    tokenized_docs = [list(chain.from_iterable(tokenized_docs[i])) for i in range(top)]
    tokenized_doc_token_slides = [list(chain.from_iterable(tokenized_doc_token_slides[i]))
                                  for i in range(top)]
    return tokenized_q, tokenized_q_token_slide,\
           tokenized_docs, tokenized_doc_token_slides,\
           docs_clean


def mark_evidence(queries, docs, hard_preds, wildcard='.'):
    wildcard_tensor = tokenizer.convert_tokens_to_ids('.') * torch.ones(max_length).type(torch.int)
    doc_max_len = max_length - 2 - len(queries[0])
    docs = [d[:doc_max_len] if len(d) >= doc_max_len 
                            else torch.cat([d, torch.zeros(doc_max_len-len(d)).type(torch.int64)])
            for d in docs]
    new_docs = []
    for q, d, e in zip(queries, docs, hard_preds):
        temp = torch.cat([torch.zeros(1).type(torch.int64), q, torch.zeros(1).type(torch.int64), d])
        temp = e * temp + (1-e) * wildcard_tensor
        new_docs.append(temp[(len(queries[0]) + 2):].type(torch.int64))
    return queries, new_docs


def merge_subtoken_exp_preds(exp_preds, slides):
    ret = []
    for p, ss in zip(exp_preds, slides):
        p = p.tolist()
        ret.append([max(p[s[0]:s[1]] + [0]) for s in ss])
    return ret


def color_cls_pred(c, 
                   pos_label='SUPPORTS', pos_color='green', 
                   neg_label='REFUTES', neg_color='red',
                   default_color='gray'):
    color = default_color
    if c == pos_label:
        color = pos_color
    elif c == neg_label:
        color = neg_color
    return f'<p style="color:{color};">{c}</p>'

    
def highlight_exp_pred(exp, doc):
    ret = ''
    abrcount = 0
    abrflag = False  # for abbreviation
    for e, w in zip(exp, doc[0]):
        if e == 1:
            ret += f'<span style="background-color:tomato; float: left">{w}&nbsp;</span>'
            abrcount = 0
            abrflag = False
        else:
            if abrflag:
                continue
            abrcount += 1
            if abrcount > 4:
                abrflag = True
                ret += f'<span style="float:left">...&nbsp;</span>'
            else:
                ret += f'<span style="float:left">{w}&nbsp;</span>'
    return ret


def postprocess(cls_preds, exp_preds, docs_clean, urls):
    cls_strs = [color_cls_pred(c) for c in cls_preds]
    evi_strs = [highlight_exp_pred(exp, doc) for exp, doc in zip(exp_preds, docs_clean)]
    pred = {
        'clses': cls_strs,
        'evis': evi_strs,
        'links': urls
    }        
    return pred
    


@app.route('/prediction/<query>')
def prediction(query):
    def _predict(exp, cls, queries, docs):
        with torch.no_grad():
            exp.eval()
            aux_preds, exp_preds, att_masks = exp(queries,
                                                  [i for i in range(top)], 
                                                  docs)
            hard_exp_preds = torch.round(exp_preds)
            queries, docs = mark_evidence(queries, docs, hard_exp_preds)
            cls_preds = cls(queries, [i for i in range(top)], docs)
            aux_preds = [classes[torch.argmax(p)] for p in aux_preds]
            cls_preds = [classes[torch.argmax(p)] for p in cls_preds]
            hard_exp_preds = [p[(len(queries[0]) + 2):] for p in hard_exp_preds]
            hard_exp_preds = merge_subtoken_exp_preds(hard_exp_preds, tokenized_d_token_slides)
        return aux_preds, cls_preds, hard_exp_preds, docs_clean   
    
    wiki_urls = bing_wiki_search(query)[:top]
    orig_docs = [get_wiki_docs(url) for url in wiki_urls]
    tokenized_q, tokenized_q_token_slide, tokenized_ds, tokenized_d_token_slides, docs_clean = \
        preprocess(query, orig_docs)
    queries = [torch.tensor(tokenized_q[0], dtype=torch.long) for i in range(top)]
    docs = [torch.tensor(s, dtype=torch.long) for s in tokenized_ds]
    aux_preds, cls_preds, exp_preds, docs_clean = _predict(evi_finder, cls, queries, docs)
    pred = postprocess(cls_preds, exp_preds, docs_clean, wiki_urls)
    pred['query'] = query
    return render_template('predict.html', pred=pred)

In [6]:
app.run(host='127.0.0.1', port=8080)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Feb/2021 01:39:58] "GET /prediction/microsoft%20is%20a%20chinese%20company HTTP/1.1" 200 -
